In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sqlite3 as db
import datetime as dt
#from sklearn.metrics import mean_squared_error

In [ ]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [ ]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [ ]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings

LSTM analysis of common birds

In [ ]:
commonCore = birdList_request(StopName='EastSunsetWay',cnx=connectDB())
commonCore = commonCore[commonCore.apply(lambda x: (x['stopGroup']==4) and (x['overall']==4),axis=1)]

In [ ]:
querySpecies = 'amerob'
StopName = 'EastSunsetWay'
query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode = "{querySpecies}" AND StopName = "{StopName}";'
obsData = pd.read_sql(query,con=connectDB(),parse_dates=['obsDt'])
obsData.set_index('obsDt',inplace=True)

In [ ]:
obsDataDaily = obsData.resample('D')
dailyData = obsDataDaily.mean()

In [ ]:
def eval_forecasts(actual,predicted):
    try:
        scores = list()
        for i in range(actual.shape[1]):
            mse = mean_squared_error(actual[:,i],predicted[:,i])
            rmse = np.sqrt(mse)
            scores.append(rmse)
        s = 0
        for row in range(actual.shape[0]):
            for col in range(actual.shapre[1]):
                s+= (actual[row,col] - predicted[row,col]**2)
        score = np.sqrt(s/(actual.shape[0]*actual.shapre[1]))
    except Exception as metricExc:
        raise metricExc
    return score, scores

In [ ]:
def split_trainTest(dailyObs):
    train, test = np.array(dailyObs[0:-328]),np.array(dailyObs[-328:-6])
    train = np.array(np.split(train,len(train)/7))
    test = np.array(np.split(test,len(test)/7))
    return train, test

In [ ]:
train, test = split_trainTest(dailyObs=dailyData)
print(test.shape)